<a href="https://colab.research.google.com/github/eorellan693/DAEN500_F2020/blob/master/VerticalSep_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and load the required Libraries 

In [7]:
#Install libraries
!pip install geopandas
!pip install geopy
!pip install shapely 
!pip install pandasql
!pip install folium

#Needed on Google Colab not AWS
!pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 15.9 MB/s 
     |████████████████████████████████| 6.3 MB 48.3 MB/s 
     |████████████████████████████████| 16.7 MB 431 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=514661c789a3cb3d6a6f616045a700a00deaf9d5b8241399def455461f3473f5
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simpl

In [8]:
#Import Libraries
import pandas as pd
import geopandas as ps
import geopy as gy
import shapely as sy
import dask.dataframe as dd
import pandasql as pq
from shapely.geometry import Point, Polygon
import numpy as np
from math import radians, cos, sin, asin, sqrt
from branca.element import Figure
import folium

#Specific libraries to Google Colab -- will not be needed in final product
from google.colab import files
import io
import os
import sys
import re

Upload Data (RawData_v2.csv file)

In [9]:
#Upload the full datafile specific to Google Drive, AWS will be different 

uploadedfile = files.upload()
rawData_df = pd.read_csv(io.BytesIO(uploadedfile['RawData_v2.csv']))
print(f'Total record count : ',len(rawData_df.index))

Saving RawData_v2.csv to RawData_v2.csv
Total record count :  33779


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Data clean up / transformation

In [10]:
#Function to filter out the needed attribues, rename, change flight level scale, and filter for only those flights at or above flight level 240
def filterAttributes():
    #New dataframe with selected attributes from the raw data
    airspaceData_df = rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong",
                     "FRN145FLFlightLevel", "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                     "FRN161TNTrackNumber"]]

    #Rename columns to make it easier to read
    airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                      'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                      'FRN131HRPWCFloatingPointLong': "Longitude", 
                                                      'FRN145FLFlightLevel': "FlightLevel", 
                                                      'FRN170TITargetId': "TargetID", 
                                                      'RESHSelectedHeading': "SelectedHeading", 
                                                      'FRN80TATargetAddress': "TargetAddress",
                                                      'FRN161TNTrackNumber': "TrackNumber"})
    
    
    #Change flight level scale to feet (FL1 = 100 ft)
    airspaceData_df['FlightLevel'] = airspaceData_df['FlightLevel'].apply(lambda x: x * 100)
    
    airspaceData = airspaceData_df
    
    return airspaceData

In [11]:
#Function to format date and time  

def timeFormatting():
    
    #Set the dataframe that will be altered through this block of code
    global allAircraftData
    
    char = ['T','Z']
    for x in char:
        allAircraftData["DateTime"] = allAircraftData["DateTime"].str.replace( x ," ")

    # Formatted Datetime
    allAircraftData["DateTime"] = pd.to_datetime(allAircraftData["DateTime"], format="%Y-%m-%d %H:%M:%S")
    
    # Create 4 new columns for Hour, Minute, Second and Microsecond
    allAircraftData["Hour"] = allAircraftData["DateTime"].dt.hour
    allAircraftData["Minute"] = allAircraftData["DateTime"].dt.minute
    allAircraftData["Second"] = allAircraftData["DateTime"].dt.second
    allAircraftData["Day"] = allAircraftData["DateTime"].dt.strftime('%Y-%m-%d')
    #allAircraftData["microSecond"] = allAircraftData["DateTime"].dt.microsecond
    
    # Reorder columns
    allAircraftData = allAircraftData[["DateTime","Day","Hour","Minute","Second","Latitude","Longitude","FlightLevel",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]

In [12]:
def dataFiltering():

  global allAircraftData

  #Remove anything below FL240
  airspaceData = allAircraftData[(allAircraftData['FlightLevel'] >= 24000)]

  #Keep only records for the first 5 seconds to speed up processing time 
  airspaceData = airspaceData[(airspaceData['Second'] < 5)]

  return airspaceData

In [13]:
#Function to filter out anything in the Hawaii airspace

def removeHISpace():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    #Coordinates for Hawaii airspace
    v0 = (26.14472222, -158.62194444) 
    v1 = (26.105, -160.63166667)
    v2 = (25.67611111, -161.69111111)
    v3 = (25.05666667, -162.64972222)
    v4 = (24.16889, -163.26638889)
    v5 = (23.25833, -163.855)
    v6 = (22.20555556, -163.91444444)

    #Select the correct v7 depending on what you are testing
    v7 = (33.10266389, 130.47177778) #Incorrect point to use during development
    #v7 = (21.1511111, -163.9144444) #Correct point to use when going live
    
    v8 = (20.11666667, -163.3)
    v9 = (19.65805556,-162.69944444)
    v10 = (19.415, -162.38361111)
    v11 = (18.40777778, -160.81416667)
    v12 = (18.0525, -160.26972222)
    v13 = (17.75583333, -159.53888889)
    v14 = (17.17055556, -157.75666667) 
    v15 = (17.805,-156.06805556)
    v16 = (18.10888889, -155.71166667)
    v17 = (19.14222222, -154.48333333)
    v18 = (19.22293333, -151.87963333)
    v19 = (20.69694444, -151.01916667) 
    v20 = (21.54777778, -151.46638889)
    v21 = (22.34416667,-151.88527778)
    v22 = (23.02416667, -152.57777778)
    v23 = (23.78055556, -153.36611111)
    v24 = (24.29583333, -154.25)
    v25 = (24.72138889, -155.26305556)
    v26 = (25.19583333, -156.42111111)

    # Polygon
    coords = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23, v24, v25, v26]
    poly = Polygon(coords)
    
    #Sort flights into what is in the airspace and what is not
    hawaiiAir = []

    for loc in range(0,len(airspaceData)):
      p1 = Point(airspaceData.iloc[loc][5], airspaceData.iloc[loc][6])
      hawaiiAir.append(p1.within(poly))

    airspaceData['nearHawaii'] = hawaiiAir
    
    #Filter out only the ones in the airspace
    airspaceData = airspaceData[(airspaceData['nearHawaii'] == False)]
    airspaceData = airspaceData.drop(columns=['nearHawaii'])
        

In [14]:
#Function to set the direction of aircraft

def aircraftDirection():
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    # Replace missing value with -1
    airspaceData['SelectedHeading'] = airspaceData['SelectedHeading'].fillna(-1)
    
    # Assign Direction "E" for 0-180 degree, "W" for 180-360 degree, "NA" is record with null values 
    conditionlist = [
        (airspaceData['SelectedHeading'] < 0) ,
        (airspaceData['SelectedHeading'] >= 0) & (airspaceData['SelectedHeading'] <180),
        (airspaceData['SelectedHeading'] > 180)]
    choicelist = ['NA', 'E', 'W']
    airspaceData['Direction'] = np.select(conditionlist, choicelist)

In [15]:
#Functions to call for the data cleanup

#Save raw data with the time formatting
allAircraftData = filterAttributes()
timeFormatting()
airspaceData = dataFiltering()
removeHISpace()
aircraftDirection()

In [16]:
airspaceData.head(10)

,DateTime,Day,Hour,Minute,Second,Latitude,Longitude,FlightLevel,TargetID,SelectedHeading,TargetAddress,TrackNumber,Direction
3,2021-12-24 04:13:04.078,2021-12-24,4,13,4,33.961899,-124.677897,34000.0,DAL495,56.25,A789BC,1486.0,E
16,2021-12-24 04:12:01.977,2021-12-24,4,12,1,33.928390,-124.854967,34000.0,DAL495,56.25,A789BC,1486.0,E
28,2021-12-24 04:11:04.156,2021-12-24,4,11,4,33.897896,-125.018292,34000.0,DAL495,56.25,A789BC,1486.0,E
42,2021-12-24 04:10:02.852,2021-12-24,4,10,2,33.865543,-125.191155,34000.0,DAL495,56.25,A789BC,1486.0,E
55,2021-12-24 04:09:00.836,2021-12-24,4,9,0,33.832165,-125.367851,34000.0,DAL495,56.25,A789BC,1486.0,E
67,2021-12-24 04:08:01.953,2021-12-24,4,8,1,33.800090,-125.536537,34000.0,DAL495,56.25,A789BC,1486.0,E
79,2021-12-24 04:07:02.188,2021-12-24,4,7,2,33.767486,-125.707640,34000.0,DAL495,56.25,A789BC,1024.0,E
92,2021-12-24 04:06:02.930,2021-12-24,4,6,2,33.734684,-125.876656,34000.0,DAL495,56.25,A789BC,1024.0,E
105,2021-12-24 04:05:02.578,2021-12-24,4,5,2,33.701248,-126.048217,34000.0,DAL495,56.25,A789BC,1024.0,E
118,2021-12-24 04:04:01.172,2021-12-24,4,4,1,33.667090,-126.222324,34000.0,DAL495,56.25,A789BC,1024.0,E


Filter out the minute intervals

In [17]:
def minuteFilter(HourCounter,MinuteCounter):

    global airspaceData

    #create SQL query for flights between the start and end time
    sql1 = "SELECT *, min(Second) FROM airspaceData WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

    #Drop the data no longer needed to speed up processing
    airspaceData = airspaceData.drop(airspaceData[(airspaceData.Hour < HourCounter) & (airspaceData.Minute > MinuteCounter)].index)

    #Run query and store results
    recordsInMinute = pq.sqldf(sql1, globals())
    del recordsInMinute['min(Second)']

    return (recordsInMinute.sort_values('Longitude').reset_index(drop=True))

Proximity Calculation

In [18]:
# Implement the formula below
def distance_d(point0,pointX):
    # The function "radians" is found in the math module
    LoA = radians(point0[1])  
    LoB = radians(pointX[1])
    LaA=  radians(point0[0])  
    LaB = radians(pointX[0]) 
    # The "Haversine formula" is used.
    D_Lo = LoB - LoA 
    D_La = LaB - LaA 
    P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))   
    # The earth's radius in kilometers.
    R_km = 6371  
 
    # Change the kilometer to  nautical miles
    R_nm = R_km*0.539956803

    # Then we'll compute the outcome.
    return(Q * R_nm)

In [19]:
# Create function to set up boundary within 25 nm by latitude, longitude 
def limit_lon(point0):
    '''
    use with LongitudeOrderDF
    note: distance from point to longitude boundary of each row is around 24.9715
    '''
    LaA = radians(point0[0])
    onedeg_long = cos(LaA)*(69.172*0.868976242)
    add = 25/onedeg_long 
    pointlimit = (point0[0],point0[1]+add)
    return pointlimit[1]

In [20]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Day','Hour','Minute','Second','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    B = OrderDF.loc[y,['DateTime','Day','Hour','Minute','Second','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [21]:
#Calculate the distance of the points closest to each other by longitidue and latitude
def proximityCalc(LongitudeOrderDF):
    longpoint_a = []
    longpoint_b = []
    longdistance_ab = []

    for a in LongitudeOrderDF.index:
        for n in range(1,len(LongitudeOrderDF)):
            b = a+n
            if b < len(LongitudeOrderDF):
                point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
                pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']
                if pointX[1] <= limit_lon(point0): # Check if longitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        longpoint_a.append(a)
                        longpoint_b.append(b)
                        longdistance_ab.append(distance)
                    else:
                        break
        
    # Apply function to select and merge data frame
    Resultsdf = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)

    # Delete duplicate pairs of TargetID x and y regardless of order
    Resultsdf['list_target'] = Resultsdf.apply(lambda row: tuple(sorted([row['TargetID_x']]+[row['TargetID_y']])), axis = 1)
    ResultsDF = Resultsdf.drop_duplicates(subset = 'list_target',keep = 'first').reset_index(drop = True)
    ResultsDF.drop('list_target', axis=1, inplace=True)

    return (ResultsDF)

Calculatin the height difference

In [22]:
def distanceCalc(resultsDF):
    heightDifference = []
    potentialLoss1000 = []
    potentialLoss400 = []

    for counter in range(0,len(resultsDF)):
        difference = abs((resultsDF['FlightLevel_x'][counter]) - (resultsDF['FlightLevel_y'][counter]))
        heightDifference.append(difference)

        if difference <= 1000:
            potentialLoss1000.append('True')
            if difference <= 400:
                potentialLoss400.append('True')
            else:
                potentialLoss400.append('False')
        else:
            potentialLoss1000.append('False')
            potentialLoss400.append('False')

    resultsDF['HeightDifference_ft'] = heightDifference
    resultsDF['potentialLoss400'] = potentialLoss400
    resultsDF['potentialLoss1000'] = potentialLoss1000

    return (resultsDF)

Compile the full list of results

In [23]:
finalResults = pd.DataFrame()

for HourCounter in range(0,1):
    #Create table for the minute
    for MinuteCounter in range(0,60):
        #Create table for the minute
        recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)

        #calculate proximity
        resultsDF = proximityCalc(recordsByMinuteDF)
        
        if resultsDF.empty == True:
            # if the results dataframe is empty, then break out of for-loop
            break
        else:
            #Calculate distance
            resultsDF = distanceCalc(resultsDF)
            #Add the results for this minute to the overall results 
            finalResults = pd.concat([finalResults, resultsDF], ignore_index=True)

In [24]:
finalResults

,DateTime_x,Day_x,Hour_x,Minute_x,Second_x,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,...,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Direction_y,Distance,HeightDifference_ft,potentialLoss400,potentialLoss1000
0,2021-12-24 00:00:02.984000,2021-12-24,0,0,2,20.830261,-156.335645,32000.0,ASA817,257.34375,...,-156.083303,29575.0,UAL1224,255.937500,AA3AE4,W,14.204775,2425.0,False,False
1,2021-12-24 00:00:03.703000,2021-12-24,0,0,3,41.108279,-154.727513,34000.0,CPA885,-1.00000,...,-154.498718,41000.0,JAL65,260.156250,86DD08,W,10.350489,7000.0,False,False
2,2021-12-24 00:01:00.742000,2021-12-24,0,1,0,20.818161,-156.453802,31725.0,ASA817,257.34375,...,-156.197728,28025.0,UAL1224,255.937500,AA3AE4,W,14.421866,3700.0,False,False
3,2021-12-24 00:01:04.188000,2021-12-24,0,1,4,41.108559,-154.903856,34000.0,CPA885,-1.00000,...,-154.673889,41000.0,JAL65,260.156250,86DD08,W,10.403371,7000.0,False,False
4,2021-12-24 00:02:02.734000,2021-12-24,0,2,2,20.805359,-156.577498,31075.0,ASA817,257.34375,...,-156.313077,28000.0,UAL1224,255.937500,AA3AE4,W,14.899400,3075.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2021-12-24 00:55:00.992000,2021-12-24,0,55,0,41.106464,-164.296854,34000.0,CPA885,-1.00000,...,-163.818325,41000.0,JAL65,258.046875,86DD08,W,21.650492,7000.0,False,False
67,2021-12-24 00:56:00.273000,2021-12-24,0,56,0,41.107395,-164.470700,34000.0,CPA885,-1.00000,...,-163.990548,41000.0,JAL65,258.046875,86DD08,W,21.723073,7000.0,False,False
68,2021-12-24 00:57:01.430000,2021-12-24,0,57,1,41.108093,-164.649719,34000.0,CPA885,-1.00000,...,-164.151794,40975.0,JAL65,258.046875,86DD08,W,22.526440,6975.0,False,False
69,2021-12-24 00:58:03.133000,2021-12-24,0,58,3,41.108551,-164.830566,34950.0,CPA885,-1.00000,...,-164.316101,41000.0,JAL65,258.046875,86DD08,W,23.274224,6050.0,False,False


List of instances that had less than 400 height difference

In [31]:
#Get the table
LossCandidates400 = finalResults.loc[(finalResults['potentialLoss400'] == 'True')]

#Add an ID to the table
LossCandidates400['LossID'] = list(range(1,len(LossCandidates400.index)+1))

#Format the table
column_to_move = LossCandidates400.pop("LossID")
LossCandidates400.insert(0, "LossID", column_to_move)

#LossCandidates400 = LossCandidates400.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
LossCandidates400.style.hide_index()

LossID,DateTime_x,Day_x,Hour_x,Minute_x,Second_x,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,TargetAddress_x,Direction_x,DateTime_y,Day_y,Hour_y,Minute_y,Second_y,Latitude_y,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Direction_y,Distance,HeightDifference_ft,potentialLoss400,potentialLoss1000
1,2021-12-24 00:24:00.531000,2021-12-24,0,24,0,22.298813,-155.816895,30000.000000,DAL495,45.703125,A789BC,E,2021-12-24 00:24:02.047000,2021-12-24,0,24,2,22.278488,-155.472823,29800.000000,AAL432,78.046875,AB271F,E,19.153655,200.000000,True,True
2,2021-12-24 00:45:00.875000,2021-12-24,0,45,0,21.387222,-155.755821,30575.000000,SWA1385,196.875000,AB5F00,W,2021-12-24 00:45:03.102000,2021-12-24,0,45,3,21.663071,-155.441362,30475.000000,SWA1310,68.203125,ABFD8C,E,24.140767,100.000000,True,True


New code to do analysis on the LossCandidates400 table
-----------------------------------------------------------

In [45]:
def recordsTable(x):

  global LossCandidates400
  
  LossCandidates400 = LossCandidates400.reset_index()

  flight_x = LossCandidates400['TargetID_x'][x]
  flight_y = LossCandidates400['TargetID_y'][x]
  hour = LossCandidates400['Hour_x'][x]
  minute = LossCandidates400['Minute_x'][x]

  flightInformation = allAircraftData.loc[((allAircraftData['TargetID'] == flight_x) | (allAircraftData['TargetID'] == flight_y)) & 
                                          ((allAircraftData['Minute'] >= (minute - 3)) & (allAircraftData['Minute'] <= (minute + 3))) & 
                                          ((allAircraftData['Hour'] == hour))]
  
  flightInformation = flightInformation.assign(SeparationEntry=x)

  return flightInformation.sort_values(by=['TargetID','Minute', 'Second'])              


In [34]:
# function for fill missing second with linear interpolation
def fillSecond(data_x,data_y):
  '''This function transform data of target y to be 
      on the same minute, second as target x'''
  # filled with NA in data_y if second_x are not in second_y 
  Y = data_y.groupby('Minute')['Second'].apply(list).reset_index(name='list')
  for i in data_x.index:
    min_x = data_x.loc[i,'Minute']
    sec_x = data_x.loc[i,'Second']
    for n in range(0,len(Y)):
      min_y = Y.loc[n,'Minute']
      if min_x == min_y:
        listsec = Y.loc[n,'list']
        if (sec_x not in listsec):
          ydict = {'Minute': min_x, 'Second': sec_x, 
                   'TargetID': data_y.loc[i,'TargetID'],
                   'SelectedHeading': data_y.loc[i,'SelectedHeading']}
          data_y = data_y.append(ydict, ignore_index = True)

  # fill NA with linear interpolation method
  y_transformed = data_y.sort_values(by=['Minute','Second']).interpolate(method='linear')
  return y_transformed

In [35]:
def transformTable(flightData):
  for i, id in enumerate(flightData['TargetID'].unique()):
    if i == 0:
      data_x = flightData[(flightData['TargetID']== id)].reset_index(drop = True)
    else:
      data_y = flightData[(flightData['TargetID']== id)].reset_index(drop = True)

  data_x = data_x[['SeparationEntry','DateTime','Day','Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]
  data_y = data_y[['Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]

  y_transformed = fillSecond(data_x,data_y)

  analyzedTable = pd.merge(data_x,y_transformed,on=['Minute','Second'], how='left')

  return analyzedTable


In [36]:
# Append the lateral Distance to table
def haversineAnalysis(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):

    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))  * 0.539956803 

In [37]:
def getLateralDist(analyzedTable):

  analyzedTable['LateralDistance'] = \
    haversineAnalysis(analyzedTable.Latitude_x, analyzedTable.Longitude_x,
                 analyzedTable.Latitude_y, analyzedTable.Longitude_y)
  
  return analyzedTable
  

In [38]:
#Append the Flight Level differnece column 
def flightlevelCalc(analyzedTable):

    flightlevelDifference = []

    for counter in range(0,len(analyzedTable)):
        Diff = abs((analyzedTable['FlightLevel_x'][counter]) - (analyzedTable['FlightLevel_y'][counter]))
        flightlevelDifference.append(Diff)

    
    analyzedTable['FlightLevelDifference'] = flightlevelDifference

    return analyzedTable


In [39]:
def getDirection(analyzedTable):

  # Direction
  conditionsX = [
      (analyzedTable.iloc[-1]['Longitude_x'] - analyzedTable.iloc[0]['Longitude_x'] < 0),
      (analyzedTable.iloc[-1]['Longitude_x'] - analyzedTable.iloc[0]['Longitude_x'] > 0)
      ]

  # create a list of the values we want to assign for each condition
  values = ['W', 'E']

  # create a new column and use np.select to assign values to it using our lists as arguments
  analyzedTable['X_direction'] = np.select(conditionsX, values)

  conditionsY = [
      (analyzedTable.iloc[-1]['Longitude_y'] - analyzedTable.iloc[0]['Longitude_y'] < 0),
      (analyzedTable.iloc[-1]['Longitude_y'] - analyzedTable.iloc[0]['Longitude_y'] > 0)
      ]

  # create a list of the values we want to assign for each condition
  values = ['W', 'E']

  # create a new column and use np.select to assign values to it using our lists as arguments
  analyzedTable['Y_direction'] = np.select(conditionsY, values)

  analyzedTable = analyzedTable[analyzedTable.columns[[0,1,2,3,4,5,6,7,8,9,17,10,11,12,13,14,18,15,16]]]

  return analyzedTable


In [46]:
analysisResults400 = pd.DataFrame()

for x in range(0,len(LossCandidates400.index)):
  #Get the data for the flight at +/- 3 minutes from when the loss of separation was flagged to be under 400 ft
  flightData = recordsTable(x)

  #Format the table for output
  analyzedTable = transformTable(flightData)

  #Compute/assign lateral separation, height separation, and direction
  analyzedTable = getLateralDist(analyzedTable)
  analyzedTable = flightlevelCalc(analyzedTable)
  analyzedTable = getDirection(analyzedTable)

  #Add table to the results 
  analysisResults400 = pd.concat([analysisResults400, analyzedTable], ignore_index=True)

analysisResults400

,SeparationEntry,DateTime,Day,Minute,Second,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,X_direction,Latitude_y,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,Y_direction,LateralDistance,FlightLevelDifference
0,0,2021-12-24 00:21:03.086,2021-12-24,21,3,22.581848,-155.291449,32600.0,AAL432,78.046875,W,22.083497,-156.167196,27912.5,DAL495,45.703125,E,57.102967,4687.5
1,0,2021-12-24 00:21:07.578,2021-12-24,21,7,22.574111,-155.296122,32525.0,AAL432,78.046875,W,22.088548,-156.159007,28025.0,DAL495,45.703125,E,56.093319,4500.0
2,0,2021-12-24 00:21:12.742,2021-12-24,21,12,22.565127,-155.301514,32450.0,AAL432,78.046875,W,22.093692,-156.150685,28100.0,DAL495,45.703125,E,55.003347,4350.0
3,0,2021-12-24 00:21:17.797,2021-12-24,21,17,22.556468,-155.306702,32375.0,AAL432,78.046875,W,22.098976,-156.142089,28137.5,DAL495,45.703125,E,53.917652,4237.5
4,0,2021-12-24 00:21:22.289,2021-12-24,21,22,22.548706,-155.311374,32300.0,AAL432,78.046875,W,22.104120,-156.133728,28175.0,DAL495,45.703125,E,52.901232,4125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1,2021-12-24 00:48:42.195,2021-12-24,48,42,21.773346,-154.893494,33000.0,SWA1310,68.203125,E,20.991056,-155.948306,21362.5,SWA1385,196.875000,W,75.390506,11637.5
173,1,2021-12-24 00:48:46.258,2021-12-24,48,46,21.775368,-154.883321,33000.0,SWA1310,68.203125,E,20.983666,-155.951876,21175.0,SWA1385,196.875000,W,76.344387,11825.0
174,1,2021-12-24 00:48:50.945,2021-12-24,48,50,21.777742,-154.871623,33000.0,SWA1310,68.203125,E,20.975915,-155.955621,20975.0,SWA1385,196.875000,W,77.399419,12025.0
175,1,2021-12-24 00:48:55.164,2021-12-24,48,55,21.779837,-154.861043,33000.0,SWA1310,68.203125,E,20.968247,-155.959349,20775.0,SWA1385,196.875000,W,78.391227,12225.0


In [ ]:
analysisResults400.to_csv('analysisResults400.csv', encoding = 'utf-8-sig') 
files.download('analysisResults400.csv')

Visualization

In [47]:
def flightXInfo(separationData):

  separationData = separationData.reset_index()

  flightX = []

  for x in range(0, len(separationData.index)):
    values_x = [separationData['DateTime'].loc[x], separationData['Latitude_x'].loc[x], separationData['Longitude_x'].loc[x], separationData['FlightLevel_x'].loc[x], 
                separationData['TargetID_x'].loc[x], separationData['X_direction'].loc[x], separationData['LateralDistance'].loc[x], separationData['FlightLevelDifference'].loc[x]]
    flightX.append(values_x)

  return flightX

In [48]:
def flightYInfo(separationData):

  separationData = separationData.reset_index()

  flightY = []

  for x in range(0, len(separationData.index)):
    values_y = [separationData['DateTime'].loc[x], separationData['Latitude_y'].loc[x], separationData['Longitude_y'].loc[x], separationData['FlightLevel_y'].loc[x], 
                separationData['TargetID_y'].loc[x], separationData['Y_direction'].loc[x], separationData['LateralDistance'].loc[x], separationData['FlightLevelDifference'].loc[x]]
    flightY.append(values_y)

  return flightY

In [49]:
def getVisTable(choice):
  
  global analysisResults400

  location = int(choice)

  separationData = analysisResults400.loc[(analysisResults400['SeparationEntry'] == (location - 1))]

  xvalues = pd.DataFrame(flightXInfo(separationData))

  yvalues = pd.DataFrame(flightYInfo(separationData))

  tableToVisualize = pd.concat([xvalues, yvalues], ignore_index=True)

  tableToVisualize = tableToVisualize.rename(columns={0: 'DateTime', 
                                                      1: "Latitude", 
                                                      2: "Longitude", 
                                                      3: "FlightLevel", 
                                                      4: "TargetID", 
                                                      5: "Direction", 
                                                      6: "LateralDistance",
                                                      7: "FLDifference"})

  return tableToVisualize

In [52]:
choice = 0

# Start a loop that runs until the user enters the value for 'quit'.
while choice != 'q':
    # Give all the choices in a series of print statements.
    print("Enter the 'LossID' of the potential loss to visualize.")
    print("")
    print("")
    print(LossCandidates400.to_string(index=False))
    print("")
    print("")
    print("Enter q to quit.")
    
    # Ask for the user's choice.
    choice = input("\nWhich would you like to view? ")
    
    # Respond to the user's choice.
    if choice != 'q':
        tableToVisualize = getVisTable(choice)
    elif choice == 'q':
        print("\nThanks for viewing the visualizations. See you later.\n")
    else:
        print("\nI don't understand that choice, please try again.\n")

Enter the 'LossID' of the potential loss to visualize.


 level_0  index  LossID                 DateTime_x      Day_x  Hour_x  Minute_x  Second_x  Latitude_x  Longitude_x  FlightLevel_x TargetID_x  SelectedHeading_x TargetAddress_x Direction_x                 DateTime_y      Day_y  Hour_y  Minute_y  Second_y  Latitude_y  Longitude_y  FlightLevel_y TargetID_y  SelectedHeading_y TargetAddress_y Direction_y  Distance  HeightDifference_ft potentialLoss400 potentialLoss1000
       0     30       1 2021-12-24 00:24:00.531000 2021-12-24       0        24         0   22.298813  -155.816895        30000.0     DAL495          45.703125          A789BC           E 2021-12-24 00:24:02.047000 2021-12-24       0        24         2   22.278488  -155.472823        29800.0     AAL432          78.046875          AB271F           E 19.153655                200.0             True              True
       1     56       2 2021-12-24 00:45:00.875000 2021-12-24       0        45         0   21.387222  -155

START INTERACTIVE BOKEH VISUALIZATION USING THE ABOVE TABLE

In [94]:
from bokeh.plotting import figure, save
import geopandas as gpd
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool

In [82]:
psource = ColumnDataSource(tableToVisualize)

psource

ColumnDataSource(id='1299', ...)

In [83]:
p = figure(title="A map of the first potential deviation")
p.circle('Latitude', 'Longitude', source=psource, color='red', size=10)

GlyphRenderer(id='1340', ...)

In [95]:
my_hover = HoverTool()

In [96]:
my_hover.tooltips = [('TimeStamp', '@DateTime'), ('TargetID', '@TargetID'), ('Latitude', '@Latitude'), ('Longitude', '@Longitude'),
                     ('FlightLevel', '@FlightLevel'), ('LateralSeparation', '@LateralDistance'), ('FlightLevelDif', '@FLDifference')]

In [97]:
p.add_tools(my_hover)

OUTPUT

In [92]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/DAEN690_VIZ/vizSep1.html'

Mounted at /content/drive


In [98]:
#outfp = r"/home/geo/data/point_map_hover.html"

path = '/content/drive/My Drive/DAEN690_VIZ/vizSep1v2.html'

save(p, path)

'/content/drive/My Drive/DAEN690_VIZ/vizSep1v2.html'

Plotting with Folium

In [99]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [137]:
from branca.element import Figure

In [209]:
list1 = []
list2 = []

for x in range(0,len(tableToVisualize.index)):
  if tableToVisualize['TargetID'][x] == 'SWA1310':
    list1.append(tableToVisualize['Latitude'][x])
    list1.append(tableToVisualize['Longitude'][x])
  else:
    list2.append(tableToVisualize['Latitude'][x])
    list2.append(tableToVisualize['Longitude'][x])

coords_1 = [list1[x:x+2] for x in range(0, len(list1), 2)]
coords_2 = [list2[x:x+2] for x in range(0, len(list2), 2)]

print(coords_1)
print(coords_2)

[[21.57258617, -155.8829753], [21.57472758, -155.8728029], [21.57673638, -155.8629873], [21.57879632, -155.8530496], [21.58232108, -155.835921], [21.58493036, -155.8233366], [21.58701746, -155.8134461], [21.58959959, -155.8009644], [21.59218473, -155.7884217], [21.59458917, -155.7767945], [21.59751881, -155.7625622], [21.60005199, -155.7503257], [21.60284517, -155.7366943], [21.60557549, -155.7233611], [21.60768656, -155.7132468], [21.61020046, -155.7008872], [21.612854, -155.688055], [21.61528006, -155.6762695], [21.61764879, -155.6649273], [21.62059017, -155.6506016], [21.62283316, -155.6396153], [21.62512193, -155.6281795], [21.62719209, -155.6181845], [21.62928756, -155.6077548], [21.63194027, -155.5947877], [21.63441461, -155.5826361], [21.63645593, -155.5725607], [21.63887663, -155.5605572], [21.64097143, -155.5504355], [21.64356982, -155.5376421], [21.64562976, -155.5274549], [21.64828012, -155.5143739], [21.65116869, -155.5001887], [21.65363364, -155.4880271], [21.65666185, -15

In [210]:
fig5=Figure(height=550,width=750)
m5=folium.Map(location=[22.298813, -155.816895],zoom_start=8)
fig5.add_child(m5)

# Creating feature groups
f1=folium.FeatureGroup("Flight1")
f2=folium.FeatureGroup("Flight2")

# Adding lines to the different feature groups
line_1=folium.vector_layers.PolyLine(coords_1,popup='<b>Path of Vehicle_1</b>',tooltip='Flight1',color='blue',weight=10).add_to(f1)
line_2=folium.vector_layers.PolyLine(coords_2,popup='<b>Path of Vehicle_2</b>',tooltip='Flight2',color='red',weight=10).add_to(f2)

f1.add_to(m5)
f2.add_to(m5)
folium.LayerControl().add_to(m5)
m5

Matplot Lib

In [53]:
tableToVisualize

,DateTime,Latitude,Longitude,FlightLevel,TargetID,Direction,LateralDistance,FLDifference
0,2021-12-24 00:42:03.633,21.572586,-155.882975,29300.0,SWA1310,E,18.327930,4700.0
1,2021-12-24 00:42:07.789,21.574728,-155.872803,29375.0,SWA1310,E,17.356377,4625.0
2,2021-12-24 00:42:11.828,21.576736,-155.862987,29425.0,SWA1310,E,16.373631,4575.0
3,2021-12-24 00:42:15.914,21.578796,-155.853050,29500.0,SWA1310,E,15.366261,4500.0
4,2021-12-24 00:42:22.891,21.582321,-155.835921,29600.0,SWA1310,E,13.791856,4400.0
...,...,...,...,...,...,...,...,...
165,2021-12-24 00:48:42.195,20.991056,-155.948306,21362.5,SWA1385,W,75.390506,11637.5
166,2021-12-24 00:48:46.258,20.983666,-155.951876,21175.0,SWA1385,W,76.344387,11825.0
167,2021-12-24 00:48:50.945,20.975915,-155.955621,20975.0,SWA1385,W,77.399419,12025.0
168,2021-12-24 00:48:55.164,20.968247,-155.959349,20775.0,SWA1385,W,78.391227,12225.0


In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [65]:
%matplotlib notebook
title = 'Flight Path'
#d = get_data(overdoses,18,title)
x = np.array(tableToVisualize['Latitude'])
y = np.array(tableToVisualize['Longitude'])
flightPath = pd.DataFrame(y,x)
#XN,YN = augment(x,y,10)
#augmented = pd.DataFrame(YN,XN)
flightPath.columns = {title}

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/animation.py", line 1467, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'


In [69]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)

In [71]:
fig = plt.figure(figsize=(10,6))
plt.xlim(18, 25)
plt.ylim(np.min(flightPath)[0], np.max(flightPath)[0])
plt.xlabel('Year',fontsize=20)
plt.ylabel(title,fontsize=20)
plt.title('Heroin Overdoses per Year',fontsize=20)

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Heroin Overdoses per Year')

In [72]:
def animate(i):
    data = flightPath.iloc[:int(i+1)] #select data range
    p = sns.lineplot(x=data.index, y=data[title], data=data, color="r")
    p.tick_params(labelsize=17)
    plt.setp(p.lines,linewidth=7)

In [73]:
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=17, repeat=True)

In [74]:
plt.show()